In [3]:
import sys
import os
import pandas as pd

# Caminho até a raiz do projeto (a pasta que contém "titulospub/")
caminho_raiz = os.path.abspath("Z:\\Chila\\projetos\\calculadora_titulos_publicos")
if caminho_raiz not in sys.path:
    sys.path.append(caminho_raiz)


from titulospub import *
from titulospub.core.auxilio import codigo_vencimento_bmf

In [4]:
vm = VariaveisMercado()
vm.atualizar_tudo()

🔄 Atualizando variáveis de mercado...
📡 Buscando feriados via scraping...
📡 Calculando IPCA dict...
📡 Buscando CDI...
📡 Realizando scraping ANBIMA...
♻️ Cache salvo para todos os títulos ANBIMA.
📡 Realizando scraping BMF...
♻️ Cache salvo para todos os contrados de DI e DAP.
📡 Realizando scraping VNA_LFT...
♻️ Cache salvo para VNA_LFT.
✅ Atualização concluída.


In [5]:
di = DI(codigo="DI1F32")
di._ajuste

✅ Usando cache existente de BMF completo.


np.float64(13.495)

In [3]:
from titulospub.core.di.calculo_di import calculo_dv01_di, taxa_pu_di

import pandas as pd

from titulospub.dados.orquestrador import VariaveisMercado
from titulospub.utils import adicionar_dias_uteis, data_vencimento_ajustada
from titulospub.core.auxilio import codigo_vencimento_bmf, vencimento_codigo_bmf


class DI:
    def __init__(self, data_vencimento: str=None,
                       codigo: str=None,
                       data_base: str=None, 
                       taxa: float=None,
                       quantidade=1, 
                       cdi: float=None,  
                       feriados: list=None,
                       variaveis_mercado: VariaveisMercado | None = None):

        # Injete uma instância para evitar recriar VariaveisMercado várias vezes
        self._vm = variaveis_mercado or VariaveisMercado()

        # Variáveis globais
        self._feriados   = feriados   if feriados   is not None else self._vm.get_feriados()
        # Datas
        self._data_base = pd.to_datetime(data_base).normalize() if data_base else pd.Timestamp.today().normalize()

        if codigo == None and data_vencimento == None:
            raise ValueError("Fornece o codigo ou vencimento")
        elif codigo == None:
            self._data_vencimento = data_vencimento_ajustada(data=pd.to_datetime(data_vencimento), feriados=self._feriados)
            self._codigo = vencimento_codigo_bmf(self._data_vencimento, "DI1")
        elif data_vencimento == None:
            self._data_vencimento = data_vencimento_ajustada(data=codigo_vencimento_bmf(codigo), feriados=self._feriados)
            self._codigo = codigo
        else:
            self._data_vencimento = data_vencimento_ajustada(data=pd.to_datetime(data_vencimento), feriados=self._feriados)
            self._codigo = codigo
        
        # Quantidade de titulos
        self._quantidade = quantidade
        self._financeiro = None  # Será calculado após _calcular()
        

        # Taxa default pela BMF do vencimento
        df_di = self._vm.get_bmf()["DI"]
        linha = df_di[df_di["DI"] == self._codigo]
        if linha.empty:
            raise ValueError(f"Vencimento {self._data_vencimento.date()} não encontrado na BMF.")
        self._ajuste = linha.squeeze()["ADJ"]

        self._taxa = float(taxa) if taxa is not None else float(self._ajuste)

        # Atributos DERIVADOS (serão preenchidos em _calcular)
        self._pu = None
        self._dv01 = None


        # Calcula já na criação
        self._calcular()
        
        # Calcula o financeiro após ter o pu
        self._financeiro = self._quantidade * self._pu


    @property
    def taxa(self): return self._taxa
    @taxa.setter
    def taxa(self, v):
        self._taxa = float(v)
        self._calcular()
    
    @property
    def data_base(self): return self._data_base
    @data_base.setter
    def data_base(self, v):
        self._data_base = pd.to_datetime(v).normalize()
        
        self._calcular()
    
    @property
    def quantidade(self):
        return self._quantidade

    @quantidade.setter
    def quantidade(self, v):
        if v <= 0:
            raise ValueError("Quantidade deve ser maior que zero")
            
        # Usa 1 como padrão para a primeira atribuição
        quantidade_anterior = getattr(self, "_quantidade", 1)

        # Ajusta valores para a unidade
        self._dv01 = self._dv01 / quantidade_anterior
        # Atualiza a quantidade
        self._quantidade = float(v)
        
        # Atualiza o financeiro baseado na nova quantidade
        self._financeiro = self._quantidade * self._pu
        # Reaplica multiplicação
        self._dv01 *= self._quantidade


    # -------- Propriedade financeiro --------
    @property
    def financeiro(self):
        return self._financeiro

    @financeiro.setter
    def financeiro(self, v):
        if v <= 0:
            raise ValueError("Financeiro deve ser maior que zero")
            
        if self._pu == 0:
            raise ValueError("PU não pode ser zero para calcular quantidade")
            
        # Usa 1 como padrão para a primeira atribuição
        quantidade_anterior = getattr(self, "_quantidade", 1)

        # Ajusta valores para a unidade
        self._dv01 = self._dv01 / quantidade_anterior

        # Calcula nova quantidade baseada no financeiro
        self._financeiro = float(v)
        self._quantidade = round(self._financeiro / self._pu, 6)

        # Reaplica multiplicação
        self._dv01 *= self._quantidade

    

    # -------- Método central de cálculo --------
    def _calcular(self):
       
        # guarda os derivados
        self._pu = taxa_pu_di(taxa=self._taxa,
                              codigo=self._codigo,
                              data_liquidacao=self._data_base,
                              data_vencimento=self._data_vencimento,
                              feriados=self._feriados)
        self._dv01 = calculo_dv01_di(taxa=self._taxa,
                                     codigo=self._codigo,
                                     data_liquidacao=self._data_base,
                                     data_vencimento=self._data_vencimento,
                                     feriados=self._feriados) * self._quantidade
        # Atualiza o financeiro baseado na quantidade atual
        self._financeiro = self._quantidade * self._pu
    

    # -------- Propriedades somente-leitura para derivados --------

    @property
    def pu(self): return self._pu
    @property
    def dv01(self): return self._dv01

In [8]:
dif32 = DI(codigo="DI1F32", data_vencimento="2032-01-01")

✅ Usando cache existente de BMF completo.


In [12]:
dif32._taxa

12.0

In [ ]:
curva = vm._bmf["DI"]
dv = []
for index, linha in curva.iterrows():
    di = DI(codigo=linha["DI"])
    dv.append(di._dv01)

curva["DV"] = dv
curva



✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF completo.
✅ Usando cache existente de BMF co

,DATA,DATA_VENCIMENTO,DI,ADJ,DV
0,2025-09-19,2025-10-01,DI1V25,14.902,0.240810
1,2025-09-19,2025-11-03,DI1X25,14.901,1.019047
2,2025-09-19,2025-12-01,DI1Z25,14.900,1.647115
3,2025-09-19,2026-01-02,DI1F26,14.891,2.358100
4,2025-09-19,2026-02-02,DI1G26,14.882,3.020736
5,2025-09-19,2026-03-02,DI1H26,14.855,3.577331
6,2025-09-19,2026-04-01,DI1J26,14.810,4.243721
7,2025-09-19,2026-05-04,DI1K26,14.767,4.836326
8,2025-09-19,2026-06-01,DI1M26,14.677,5.420319
9,2025-09-19,2026-07-01,DI1N26,14.586,6.021497


: 

In [ ]:


def vencimento_codigo_bmf(data_vencimento, prefixo):    
    letras  = {"01":"F", "02":"G", "03":"H",
                "04":"J", "05":"K", "06":"M",
                "07":"N", "08":"Q", "09":"U",
                "10":"V", "11":"X", "12":"Z"}

    dt = data_vencimento.strftime("%Y-%m-%d")
    ano = dt[2:4]
    mes = dt[5:7]


    for k, v in letras.items():
        mes = mes.replace(k, v)

    codigo = f"{prefixo}{mes}{ano}"
    return codigo

In [20]:
data_vencimento = pd.Timestamp.today().normalize()
prefixo = "DI1"
vencimento_codigo_bmf(data_vencimento, prefixo)

'DI1U25'

In [46]:
dvdi = calculo_dv01_di(taxa=14.055, data_vencimento=pd.to_datetime("2028-01-01"))

In [43]:
b28 = NTNB("2028-08-15", quantidade=1)

✅ Usando cache existente de ANBIMAS completo.


In [51]:
10000 * dvdi/100 / b28._dv01

np.float64(1315.4125624917763)

In [49]:
10000 *  b28._dv01 / dvdi

np.float64(760.2177662844479)

In [ ]:
curva = vm._bmf["DI"]
curva["DV"] = [DI(codigo=codigo).dv01 for codigo in curva["DI"]]


In [ ]:
# Vamos verificar o que tem na BMF para entender o erro
vm = VariaveisMercado()
df_di = vm.get_bmf()["DI"]
print("Colunas disponíveis:", df_di.columns.tolist())
print("\nPrimeiras linhas:")
print(df_di.head())
print("\nTipo de dados da coluna ADJ:")
print(df_di["ADJ"].dtype)
print("\nValores únicos da coluna ADJ:")
print(df_di["ADJ"].unique()[:10])
